In [2]:
import pandas as pd
from collections import namedtuple
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import networkx as nx


In [3]:
label_index=302

In [60]:
# # 加载数据，并转换为torch.Tensor
# trainData = pd.read_pickle('../processed/corax_train.pkl')
# testData = pd.read_pickle('../processed/corax_test.pkl')
# validationData=pd.read_pickle('../processed/corax_validation.pkl')
# allData=pd.concat([trainData,testData,validationData],axis=0)

# # trainLabel = trainData[label_index]
# # trainData = trainData.drop(columns=label_index)

# # testLabel = testData[label_index]
# # testData = testData.drop(columns=label_index)

# # validationLabel = validationData[label_index]
# # validationData = validationData.drop(columns=label_index)

# allLabel= allData[label_index]
# allData = allData.drop(columns=label_index)

# G=nx.read_gpickle('../processed/corax_graph.gpickle')


In [4]:
feature = pd.read_pickle('../corpus/corax_features.pkl')
label = pd.read_pickle('../corpus/corax_labels.pkl')
adj_matrix=pd.read_pickle('../corpus/corax_adj.pkl')

label_onehot = torch.tensor(label)
y = torch.topk(label_onehot, 1)[1].squeeze(1).numpy()

G = nx.from_numpy_matrix(adj_matrix)

In [5]:
# allLabel.values

In [6]:
from torch_geometric.data import Data

# 确定x
x = torch.tensor(feature, dtype=torch.float)

# 确定edge_index
edge_index = torch.tensor(list(G.edges)).t().contiguous()

# 确定y
y = torch.tensor(y, dtype=torch.long)

train_mask = np.zeros(x.shape[0], dtype=bool)
val_mask = np.zeros(x.shape[0], dtype=bool)
test_mask = np.zeros(x.shape[0], dtype=bool)

train_mask[0:1180]=True
val_mask[1180:1680]=True
test_mask[1680:2681]=True

data=Data(x=x,edge_index=edge_index,y=y,train_mask=train_mask,val_mask=val_mask,test_mask=test_mask)
data

Data(x=[2680, 302], edge_index=[2, 5148], y=[2680], train_mask=[2680], val_mask=[2680], test_mask=[2680])

In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = F.softmax(x, dim=1)

        return x

In [8]:
num_node_features=data.x.shape[1]
num_classes = 7
model=GCN(num_node_features,num_classes)

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
loss_function = torch.nn.CrossEntropyLoss()
model.train()
for epoch in range(200):
    out = model(data)
    optimizer.zero_grad()
    loss = loss_function(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    print('Epoch {:03d} loss {:.4f}'.format(epoch, loss.item()))
    

model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('GCN Accuracy: {:.4f}'.format(acc))

Epoch 000 loss 1.9464
Epoch 001 loss 1.9355
Epoch 002 loss 1.9186
Epoch 003 loss 1.9121
Epoch 004 loss 1.8936
Epoch 005 loss 1.8828
Epoch 006 loss 1.8909
Epoch 007 loss 1.8458
Epoch 008 loss 1.8555
Epoch 009 loss 1.8487
Epoch 010 loss 1.8612
Epoch 011 loss 1.8302
Epoch 012 loss 1.8214
Epoch 013 loss 1.8267
Epoch 014 loss 1.8027
Epoch 015 loss 1.8026
Epoch 016 loss 1.8257
Epoch 017 loss 1.8072
Epoch 018 loss 1.7912
Epoch 019 loss 1.8161
Epoch 020 loss 1.7732
Epoch 021 loss 1.8079
Epoch 022 loss 1.7949
Epoch 023 loss 1.7833
Epoch 024 loss 1.7711
Epoch 025 loss 1.7744
Epoch 026 loss 1.7852
Epoch 027 loss 1.7877
Epoch 028 loss 1.7716
Epoch 029 loss 1.7795
Epoch 030 loss 1.7684
Epoch 031 loss 1.7498
Epoch 032 loss 1.7818
Epoch 033 loss 1.7653
Epoch 034 loss 1.7684
Epoch 035 loss 1.7503
Epoch 036 loss 1.7552
Epoch 037 loss 1.7658
Epoch 038 loss 1.7616
Epoch 039 loss 1.7814
Epoch 040 loss 1.7704
Epoch 041 loss 1.7463
Epoch 042 loss 1.7625
Epoch 043 loss 1.7455
Epoch 044 loss 1.7360
Epoch 045 